In [1]:
#Install everything
#pip3 install python-osc
#pip3 install twitter
#pip3 install numpy
#pip3 install nltk

In [24]:
# Import everything
import twitter
from pythonosc.udp_client import SimpleUDPClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import json
#import numpy as np

In [25]:
# Instantiate the twitter api client
CONSUMER_KEY = 'OWAwcOfkzNSlIBGhAtQdNY4Im'
CONSUMER_SECRET = '14iE08KZpZx44b4B0WfoAqvOPF7kXNdzkkv7fmXreqKb1fCzMc'
OAUTH_TOKEN = ''
OAUTH_TOKEN_SECRET = ''

auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(auth=auth)

In [26]:
# Instantiate the OSC UDP client
ip = "127.0.0.1"
port = 57120

client = SimpleUDPClient(ip, port)  # Create OSC client


In [27]:
# Instantiate the sentiment analyzer
nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/juanrieragomez/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [28]:
# Obtain world trends
WORLD_WOE_ID = 1
US_WOE_ID = 23424977
world_trends = twitter_api.trends.place(_id=WORLD_WOE_ID)

In [29]:
# Obtain average twitter sentiment
count = 0
sentiment = {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}
for trend in world_trends[0]['trends']:
    search_results = twitter_api.search.tweets(q=trend['query'], count=100, result_type='popular')   
    keys=['neg', 'neu', 'pos', 'compound']
    for status in search_results['statuses']:
        tweet_sentiment = analyzer.polarity_scores(status['text'])
        for key in keys:
            sentiment[key] += tweet_sentiment[key]
        count +=1

for key in keys:
    sentiment[key] /= count

print(sentiment)

{'neg': 0.025540160642570288, 'neu': 0.908483935742972, 'pos': 0.06599799196787151, 'compound': 0.10194598393574301}


In [30]:
# Send the sentiment to SC
client.send_message("/BNOOSC/Sentiment/", [sentiment['neg'], sentiment['neu'], sentiment['pos'], sentiment['compound']])



In [56]:
since_id_hashtag = 0
init_tweet_text = 'init'

In [79]:
# Scan for custom Hashtags
customHashtag = '#BNODNA'
q = customHashtag' since_id:' + str(since_id_hashtag)
count = 10
search_results = twitter_api.search.tweets(q=q, count=count)
statuses = search_results['statuses']
#print(json.dumps(statuses[0], indent=1))
#for status in statuses:
#    print(status['text'])
if(len(statuses)>0):
    since_id_hashtag = statuses[0]['id']

In [80]:
# Send the customTweet to SC
for status in statuses:
    if customHashtag in status['text'] and init_tweet_text in status['text']:
        print("INIT HASHTAG FOUND " + status['created_at'])
    else:
        client.send_message("/BNOOSC/CustomHashtag/", [customHashtag, status['text'].replace(customHashtag, '').strip()])

In [85]:
# Create note strings
notes = ['#A', '#B', '#C', '#D', '#E', '#F', '#G']
flats = []
sharps = []
for note in notes:
    flats.append(note + 'f')
    sharps.append(note + 's')

['Af', 'Bf', 'Cf', 'Df', 'Ef', 'Ff', 'Gf']